In [ ]:
import os
os.environ['APOGEE_CACHE_PATH'] = "/mnt/ceph/users/apricewhelan/apogee-test/"
import warnings
warnings.filterwarnings('ignore', category=Warning) 

import sys
import pathlib
_path = str(pathlib.Path('../').resolve())
if _path not in sys.path:
    sys.path.append(_path)

import corner
from astropy.io import fits
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm.auto import tqdm
from sklearn.decomposition import IncrementalPCA

from joaquin import Joaquin
from joaquin.design_matrix import JoaquinData
from joaquin.config import phot_names, dr, Kfold_K
from joaquin.logger import logger
from joaquin.plot import simple_corner, phot_to_label

from gala.mpl_style import hesperia, laguna

In [ ]:
cache_path = pathlib.Path(f'../cache/{dr}').resolve()
cache_path.mkdir(exist_ok=True, parents=True)

In [ ]:
parent = at.Table.read(cache_path / 'parent-sample.fits')

parent_stars = parent[parent['SNR'] > 100]

# HACK: subselect for speed
np.random.seed(42)
idx = np.random.choice(len(parent_stars), size=8192, replace=False)
parent_stars = parent_stars[idx]

len(parent_stars)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

bins = (np.linspace(3000, 7500, 128),
        np.linspace(0, 5.5, 128))
ax.hist2d(parent['TEFF'], parent['LOGG'],
          bins=bins, norm=mpl.colors.LogNorm(),
          cmap='magma_r')

ax.plot(parent_stars['TEFF'],
        parent_stars['LOGG'],
        ls='none', marker='o', mew=0, ms=3., 
        color='tab:blue', alpha=0.75)

ax.set_ylim(5.5, 0)
ax.set_xlim(7500, 3000)

fig.tight_layout()

In [ ]:
data, star_mask = JoaquinData.from_stars(
    parent_stars, lowpass=False)

In [ ]:
X, *_ = data.get_sub_Xy('spec')

good_mask = np.all(np.isfinite(X), axis=1)
X = X[good_mask]
good_stars = parent_stars[good_mask]
X.shape

In [ ]:
pca = IncrementalPCA(n_components=8, batch_size=1024)
projected_X = pca.fit_transform(X)

This hacky step removes egregious outliers - only run this after the cells above because it overwrites variables!!

In [ ]:
mean = np.mean(projected_X, axis=0)
std = np.std(projected_X, axis=0)
bad_mask = np.any(np.abs(projected_X - mean) > 5*std, axis=1)

X = X[~bad_mask]
good_stars = good_stars[~bad_mask]

pca = IncrementalPCA(n_components=8, batch_size=1024)
projected_X = pca.fit_transform(X)

In [ ]:
for i in range(pca.n_components):
    fig = plt.figure(figsize=(12, 3))
    plt.plot(pca.components_[i])

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
fig, axes = plt.subplots(3, 3, 
                         figsize=(10, 10),
                         sharex=True, sharey=True)

for i in range(pca.n_components):
    ax = axes.flat[i]
    ax.scatter(good_stars['TEFF'],
               good_stars['LOGG'],
               c=projected_X[:, i], s=6)

ax.set_ylim(5.5, 0)
ax.set_xlim(7500, 3000)

fig.tight_layout()

In [ ]:
things = {
    'TEFF': (3000, 6500),
    'LOGG': (0.5, 5.5),
    'M_H': (-2, 0.5)
}

In [ ]:
for name, (vmin, vmax) in things.items():
    fig, axes = simple_corner(projected_X, 
                              color_by=good_stars[name],
                              vmin=vmin, vmax=vmax)
    axes.flat[0].set_title(f'color: {name}')

In [ ]:
np.save(cache_path / 'pca_neighborizer.npy', 
        np.vstack((pca.mean_, pca.components_)))

---

## Validation

In [ ]:
from scipy.spatial import cKDTree

In [ ]:
tree = cKDTree(projected_X / pca.singular_values_)

dist, idx = tree.query(projected_X / pca.singular_values_, 
                       k=np.arange(2, 32+2))

In [ ]:
np.std(projected_X / pca.singular_values_, axis=0)

In [ ]:
for i, js in enumerate(idx[:20]):    
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    ax = axes[0]
    bins = (np.linspace(3000, 7500, 128),
            np.linspace(0, 5.5, 128))
    ax.hist2d(parent['TEFF'], parent['LOGG'],
              bins=bins, norm=mpl.colors.LogNorm(),
              cmap='Greys')

    ax.plot(good_stars['TEFF'][i],
            good_stars['LOGG'][i],
            ls='none', marker='s', mew=0, ms=6., 
            color='tab:blue', zorder=100)

    ax.plot(good_stars['TEFF'][js],
            good_stars['LOGG'][js],
            ls='none', marker='o', mew=0, ms=4., 
            color='tab:orange', zorder=10)

    ax.set_ylim(5.5, 0)
    ax.set_xlim(7500, 3000)
    
    # ---
    
    ax = axes[1]
    
    bins = (np.linspace(-0.5, 4.5, 128),
            np.linspace(-4, 12, 128))
    dist_mask, = np.where((parent['GAIAEDR3_PARALLAX'] / parent['GAIAEDR3_PARALLAX_ERROR']) > 5)
    distmod = coord.Distance(parallax=parent['GAIAEDR3_PARALLAX'][dist_mask]*u.mas).distmod.value
    ax.hist2d((parent['GAIAEDR3_PHOT_G_MEAN_MAG'] - parent['J'])[dist_mask], 
              parent['GAIAEDR3_PHOT_G_MEAN_MAG'][dist_mask] - distmod,
              bins=bins, norm=mpl.colors.LogNorm(),
              cmap='Greys')
    
    distmod = coord.Distance(parallax=good_stars['GAIAEDR3_PARALLAX']*u.mas, 
                             allow_negative=True).distmod.value
    ax.plot((good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - good_stars['J'])[i],
            (good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - distmod)[i],
            ls='none', marker='s', mew=0, ms=6., 
            color='tab:blue', zorder=100)
    
    dist_mask = (parent['GAIAEDR3_PARALLAX'][js] > 0.5) & (parent['GAIAEDR3_PARALLAX_ERROR'][js] < 0.1)
    ax.plot((good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - good_stars['J'])[js][dist_mask],
            (good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - distmod)[js][dist_mask],
            ls='none', marker='o', mew=0, ms=4., 
            color='tab:orange', zorder=10)

    ax.set_xlim(bins[0].min(), bins[0].max())
    ax.set_ylim(bins[1].max(), bins[1].min())

    fig.tight_layout()

In [ ]:
is_ = np.arange(len(good_stars))

for i, js in zip(is_[good_stars['LOGG'] > 4], 
                 idx[good_stars['LOGG'] > 4][:20]):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    ax = axes[0]
    bins = (np.linspace(3000, 7500, 128),
            np.linspace(0, 5.5, 128))
    ax.hist2d(parent['TEFF'], parent['LOGG'],
              bins=bins, norm=mpl.colors.LogNorm(),
              cmap='Greys')

    ax.plot(good_stars['TEFF'][i],
            good_stars['LOGG'][i],
            ls='none', marker='s', mew=0, ms=6., 
            color='tab:blue', zorder=100)

    ax.plot(good_stars['TEFF'][js],
            good_stars['LOGG'][js],
            ls='none', marker='o', mew=0, ms=4., 
            color='tab:orange', zorder=10)

    ax.set_ylim(5.5, 0)
    ax.set_xlim(7500, 3000)
    
    # ---
    
    ax = axes[1]
    
    bins = (np.linspace(-0.5, 4.5, 128),
            np.linspace(-4, 12, 128))
    dist_mask, = np.where((parent['GAIAEDR3_PARALLAX'] / parent['GAIAEDR3_PARALLAX_ERROR']) > 5)
    distmod = coord.Distance(parallax=parent['GAIAEDR3_PARALLAX'][dist_mask]*u.mas).distmod.value
    ax.hist2d((parent['GAIAEDR3_PHOT_G_MEAN_MAG'] - parent['J'])[dist_mask], 
              parent['GAIAEDR3_PHOT_G_MEAN_MAG'][dist_mask] - distmod,
              bins=bins, norm=mpl.colors.LogNorm(),
              cmap='Greys')
    
    distmod = coord.Distance(parallax=good_stars['GAIAEDR3_PARALLAX']*u.mas, 
                             allow_negative=True).distmod.value
    ax.plot((good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - good_stars['J'])[i],
            (good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - distmod)[i],
            ls='none', marker='s', mew=0, ms=6., 
            color='tab:blue', zorder=100)

    dist_mask = (parent['GAIAEDR3_PARALLAX'][js] > 0.5) & (parent['GAIAEDR3_PARALLAX_ERROR'][js] < 0.1)
    ax.plot((good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - good_stars['J'])[js][dist_mask],
            (good_stars['GAIAEDR3_PHOT_G_MEAN_MAG'] - distmod)[js][dist_mask],
            ls='none', marker='o', mew=0, ms=4., 
            color='tab:orange', zorder=10)

    ax.set_xlim(bins[0].min(), bins[0].max())
    ax.set_ylim(bins[1].max(), bins[1].min())

    fig.tight_layout()